In [1]:
import pandas as pd
import geopandas as gpd
import os
import ast
import numpy as np
import math

pd.set_option('display.max_columns', None)
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())

C:\Users\LMENENDEZ\GitHub\MultiModX


In [2]:
%load_ext autoreload

In [3]:
%autoreload
from script.trips_format import *

## 1. Download AW data

In [4]:
aw_paths_w_costs_airports=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\aw\aw_paths_costs.csv")

aw_paths_w_costs_airports has information about the city of origin, the city of destination, the air path taken and the costs of this path. The path is a sequence of airports.

In [5]:
aw_paths_w_costs_airports.sample(4)

,Unnamed: 0,origin,destination,origin_city,destination_city,path,Passengers,origin_airport,destination_airport,Avg. Total Fare(USD),Distance (km)
223,223,EDDS,GCRR,EDDS,GCRR,"['EDDS', 'GCRR']",50.759667,EDDS,GCRR,113.29,2932.0
785,785,ENTO,LEMG,OSLO,LEMG,"['ENTO', 'LEMG']",152.316000,ENTO,LEMG,80.47,2720.0
2536,2536,LEST,LFSB,LEST,LFSB,"['LEST', 'LFSB']",64.946333,LEST,LFSB,101.50,1350.0
756,756,ELLX,LEMD,ELLX,LEMD,"['ELLX', 'LEMD']",169.562667,ELLX,LEMD,131.48,1273.0


In [6]:
def format_aw_paths(paths):
    paths=paths.drop(["Unnamed: 0"],axis=1) #drops additional column
    paths["cost"]=round(paths["Avg. Total Fare(USD)"]*0.9,2) #gets prices in euro

    paths=paths.dropna() #drop nan values

    
    # co2 emissions should be added at some point
    return paths


In [7]:
aw_paths_w_costs_airports=format_aw_paths(aw_paths_w_costs_airports)

In [8]:
aw_paths_w_costs_airports.head()

,origin,destination,origin_city,destination_city,path,Passengers,origin_airport,destination_airport,Avg. Total Fare(USD),Distance (km),cost
0,BIKF,GCLP,BIKF,GCLP,"['BIKF', 'GCLP']",24.100000,BIKF,GCLP,149.34,4041.0,134.41
1,BIKF,GCTS,BIKF,GCTS,"['BIKF', 'GCTS']",74.633333,BIKF,GCTS,148.93,4020.0,134.04
2,BIKF,LEAL,BIKF,LEAL,"['BIKF', 'LEAL']",123.818333,BIKF,LEAL,161.65,3207.0,145.49
3,BIKF,LEBL,BIKF,LEBL,"['BIKF', 'LEBL']",136.487000,BIKF,LEBL,198.16,2981.0,178.34
4,BIKF,LEMD,BIKF,LEMD,"['BIKF', 'LEMD']",100.200333,BIKF,LEMD,216.87,2892.0,195.18


In [9]:
def create_aw_paths_cities(paths):
    #creates a new columns
    paths["list_of_nodes"]=paths["path"].apply(lambda x: ast.literal_eval(x))

    #creates two masks
    mask_origin=paths["origin_city"]!=paths["list_of_nodes"].str[0]
    mask_destination=paths["destination_city"]!=paths["list_of_nodes"].str[-1]

    #changes first node to origin city
    paths.loc[mask_origin,"list_of_nodes"]=paths[mask_origin].apply(
        lambda row: [row["origin_city"]]+row["list_of_nodes"][1:],axis=1
    )

    #changes last node to destination city
    paths.loc[mask_destination,"list_of_nodes"]=paths[mask_destination].apply(
        lambda row: row["list_of_nodes"][:-1]+[row["destination_city"]],axis=1
    )
    # update the column path
    paths["path"]=paths["list_of_nodes"].astype(str)

    #drop the list_of_nodes columns
    paths=paths.drop(["list_of_nodes"],axis=1)

    #groupby for each path and take average of the rest of 
    columns=list(paths.columns)
    columns_grouped_by=[column for column in columns if column not in ["Avg. Total Fare(USD)","Distance (km)","cost", "Passengers"]]
    total_before = paths[paths.notna()]["Passengers"].sum()

    paths["Passengers"]=paths["Passengers"].round(6)
    paths=paths.groupby(columns_grouped_by).agg({
        "Avg. Total Fare(USD)":"mean",
        "Distance (km)": "mean",
        "cost":"mean",
        "Passengers": "sum"
    }).reset_index()
    
    total_after = paths.groupby(columns_grouped_by)["Passengers"].sum().sum()
    print(f"Sum before groupby: {total_before}, after: {total_after}")
    return paths


In [10]:
aw_paths_w_costs_cities=create_aw_paths_cities(aw_paths_w_costs_airports)

Sum before groupby: 528112.7383333333, after: 528112.738316


In [11]:
aw_paths_w_costs_cities.sample(4)

,origin,destination,origin_city,destination_city,path,origin_airport,destination_airport,Avg. Total Fare(USD),Distance (km),cost,Passengers
1836,LEGE,UUDD,LEGE,MOSCOW,"['LEGE', 'MOSCOW']",LEGE,UUDD,127.97,2940.0,115.17,14.033333
881,ESNN,LEPA,ESNN,LEPA,"['ESNN', 'LEPA']",ESNN,LEPA,124.07,2739.0,111.66,30.066667
2284,LEMH,LFPO,LEMH,PARIS,"['LEMH', 'PARIS']",LEMH,LFPO,98.56,996.0,88.70,44.478000
1200,GCTS,EIDW,GCTS,EIDW,"['GCTS', 'EIDW']",GCTS,EIDW,140.44,2945.0,126.40,338.032000


## 2. Download the coefficients

In [12]:
coeff_outgoing=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\coefficients\coefficients_outgoing_trips_from_spain_v0.5.csv")

In [13]:
coeff_incoming=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\coefficients\coefficients_incoming_trips_to_spain_v0.5.csv")

In [14]:
coeff_incoming.columns

Index(['last_airport', 'destination', 'destination_name',
       'node_sequence_ground', 'trips', 'total_trips', 'coeff'],
      dtype='object')

In [15]:
aw_paths_w_costs_small=aw_paths_w_costs_cities[(aw_paths_w_costs_cities["origin_city"]=="LEZL")|(aw_paths_w_costs_cities["destination_city"]=="LEZL")]

In [16]:
# Group by 'last_airport' and 'node_sequence_spain', then count unique 'destination' values
conflicts = (
    coeff_incoming.groupby(['last_airport', 'node_sequence_ground'])
    ['destination'].nunique()  # Count unique destinations per group
    .gt(1)                    # Check if count > 1 (conflict exists)
    .reset_index(name='has_conflict')
)

# Get the full rows where conflicts occur
conflicting_rows = coeff_incoming.merge(
    conflicts[conflicts['has_conflict']],
    on=['last_airport', 'node_sequence_ground']
)


In [17]:
conflicting_rows[conflicting_rows["node_sequence_ground"]!="egress"]

,last_airport,destination,destination_name,node_sequence_ground,trips,total_trips,coeff,has_conflict
26,LEBB,ES211,Álava,"['007113200', '007111200']",0.321482,6183.029558,0.000052,True
30,LEBB,ES220,Navarra,"['007113200', '007111200']",0.049486,6183.029558,0.000008,True
32,LEBB,ES230,La Rioja,"['007113200', '007111200']",1.355357,6183.029558,0.000219,True
36,LEBB,ES412,Burgos,"['007113200', '007111200']",8.218265,6183.029558,0.001329,True
71,LEBL,ES514,Tarragona,"['007171801', '007165402']",238.390015,73626.275263,0.003238,True
73,LEBL,ES521,Alicante / Alacant,"['007171801', '007165402']",1.184994,73626.275263,0.000016,True
75,LEBL,ES522,Castellón / Castelló,"['007171801', '007165402']",1.458899,73626.275263,0.000020,True
108,LEGE,ES511,Barcelona,"['007179300', '007171801']",467.269906,10800.651475,0.043263,True
111,LEGE,ES513,Lleida,"['007179300', '007171801']",0.953331,10800.651475,0.000088,True
113,LEGE,ES514,Tarragona,"['007171801', '007165402']",13.061762,10800.651475,0.001209,True


## 3. Assign ground paths

In [119]:
def generate_probabilities_incoming(selected_paths: pd.DataFrame):
    """given a set a options, selects and normalises the probabilities"""
    paths_options=selected_paths[["node_sequence_ground","destination","destination_name"]].values
    probs= selected_paths["coeff"].values
    probs= probs/probs.sum() #normalises probabilities

    paths_options_tuples = [tuple(row) for row in paths_options]
    return paths_options_tuples, probs

In [120]:
def generate_probabilities_outgoing(selected_paths: pd.DataFrame):
    """given a set a options, selects and normalises the probabilities"""
    paths_options=selected_paths[["node_sequence_ground","origin","origin_name"]].values
    probs= selected_paths["coeff"].values
    probs= probs/probs.sum() #normalises probabilities

    paths_options_tuples = [tuple(row) for row in paths_options]
    return paths_options_tuples, probs

In [121]:
def generate_ground_path_incoming(selected_trips: pd.DataFrame,paths_options,probs):
    """given selected trips, generates the international path using the probabilities of path_options_probs"""
    new_rows=[]
    for _, row in selected_trips.iterrows():
        trips_raw=row["Passengers"]
        trips_count = math.ceil(trips_raw) if trips_raw < 1 else round(trips_raw) # Number of trips to generate

        chosen_indices = np.random.choice(
            len(paths_options), 
            size=trips_count,  # Generate all needed samples in one call
            p=probs
        )

      # Get paths corresponding to chosen indices
        chosen_paths = [paths_options[i] for i in chosen_indices]
        
        # Count occurrences of each path
        path_counts = {}
        for path in chosen_paths:
            path_tuple = tuple(path)  # Ensure hashability
            path_counts[path_tuple] = path_counts.get(path_tuple, 0) + 1
        
        # Build output rows
        for path, count in path_counts.items():
            new_row = row.to_dict()
            new_row.update({
                "node_sequence_ground": path[0],
                "destination": path[1],
                "destination_name": path[2],
                "trips": float(count)
            })
            new_rows.append(new_row)

    return new_rows

In [122]:
def generate_ground_path_outgoing(selected_trips: pd.DataFrame,paths_options,probs):
    """given selected trips, generates the international path using the probabilities of path_options_probs"""
    new_rows=[]
    for _, row in selected_trips.iterrows():
        trips_raw=row["Passengers"]
        trips_count = math.ceil(trips_raw) if trips_raw < 1 else round(trips_raw) # Number of trips to generate

        chosen_indices = np.random.choice(
            len(paths_options), 
            size=trips_count,  # Generate all needed samples in one call
            p=probs
        )

      # Get paths corresponding to chosen indices
        chosen_paths = [paths_options[i] for i in chosen_indices]
        
        # Count occurrences of each path
        path_counts = {}
        for path in chosen_paths:
            path_tuple = tuple(path)  # Ensure hashability
            path_counts[path_tuple] = path_counts.get(path_tuple, 0) + 1
        
        # Build output rows
        for path, count in path_counts.items():
            new_row = row.to_dict()
            new_row.update({
                "node_sequence_ground": path[0],
                "origin": path[1],
                "origin_name": path[2],
                "trips": float(count)
            })
            new_rows.append(new_row)

    return new_rows

In [123]:
coeff_incoming.columns.values

array(['last_airport', 'destination', 'destination_name',
       'node_sequence_ground', 'trips', 'total_trips', 'coeff'],
      dtype=object)

In [166]:
def assign_ground_path(trips_aw: pd.DataFrame, coeff_outgoing: pd.DataFrame, coeff_incoming: pd.DataFrame):
        #create two empty dataframe with the relevant columns
        columns_aw=list(trips_aw.columns.values)
        columns_in=[column for column in coeff_incoming.columns if column not in ["total_trips","coeff"]]
        columns_out=[column for column in coeff_incoming.columns if column not in ["total_trips","coeff"] ]
        columns_selected=list(set(columns_aw+columns_in+columns_out))
        trips_incoming_with_complete_path=pd.DataFrame(columns=columns_selected)
        trips_outgoing_with_complete_path=pd.DataFrame(columns=columns_selected)
        
        #create lists of spanish origin and destination airports
        destination_airports=list(trips_aw["destination_airport"])
        spanish_destination_airports=set([airport for airport in destination_airports if airport.startswith("LE") or airport.startswith("GC")])
        origin_airports=list(trips_aw["origin_airport"])
        spanish_origin_airports=set([airport for airport in origin_airports if airport.startswith("LE") or airport.startswith("GC")])

        # first iterate over destination airports (incoming trips)
        for airport in spanish_destination_airports:
                selected_trips=trips_aw[trips_aw["destination_airport"]==airport]
                selected_ground_paths=coeff_incoming[coeff_incoming["last_airport"]==airport]
                if not selected_ground_paths.empty:
                        ground_paths_options, probs=generate_probabilities_incoming(selected_ground_paths)
                        new_rows=generate_ground_path_incoming(selected_trips,ground_paths_options,probs)
                        #print(ground_paths_options)
                        #print(probs)
                        new_df=pd.DataFrame(new_rows)
                        columns= list(new_df.columns.values)
                        columns.remove("trips")
                        #print(new_rows)
                        new_df=new_df.groupby(columns).sum().reset_index()
                        trips_incoming_with_complete_path=pd.concat([trips_incoming_with_complete_path,new_df])

        # drop irrelevant columns for trips_incoming_with_complete_path
        trips_incoming_with_complete_path=trips_incoming_with_complete_path.drop(columns=["destination",
                                                                "last_airport",
                                                                "origin_airport",
                                                                "destination_airport",
                                                                "origin",
                                                                "Avg. Total Fare(USD)",
                                                                "destination_city"
                                                                ])
        
        # rename columns for trips_incoming_with_complete_path for clarity
        trips_incoming_with_complete_path=trips_incoming_with_complete_path.rename(columns={"origin_city": "origin",
                                                                          "destination_name":"destination",
                                                                          "Distance (km)":"distance_air",
                                                                          "cost":"cost_air",
                                                                          "path":"node_sequence_air",
                                                                          "Passengers":"trips_air"
                                                                          })
        # second iterate over origin airports (outgoing trips)
        for airport in spanish_origin_airports:
                selected_trips=trips_aw[trips_aw["origin_airport"]==airport]
                selected_ground_paths=coeff_outgoing[coeff_outgoing["first_airport"]==airport]
                if not selected_ground_paths.empty:
                        ground_paths_options, probs=generate_probabilities_outgoing(selected_ground_paths)
                        new_rows=generate_ground_path_outgoing(selected_trips,ground_paths_options,probs)
                        #print(ground_paths_options)
                        #print(probs)
                        new_df=pd.DataFrame(new_rows)
                        columns= list(new_df.columns.values)
                        columns.remove("trips")
                        #print(new_rows)
                        new_df=new_df.groupby(columns).sum().reset_index()
                        trips_outgoing_with_complete_path=pd.concat([trips_outgoing_with_complete_path,new_df])

        trips_outgoing_with_complete_path=trips_outgoing_with_complete_path.drop(columns=["destination",
                                                        "last_airport",
                                                        "origin_airport",
                                                        "destination_airport",
                                                        "origin",
                                                        "Avg. Total Fare(USD)",
                                                        "origin_city",
                                                        "destination_name"
                                                        ])
        
        trips_outgoing_with_complete_path=trips_outgoing_with_complete_path.rename(columns={"destination_city": "destination",
                                                                        "origin_name":"origin",
                                                                        "Distance (km)":"distance_air",
                                                                        "cost":"cost_air",
                                                                        "path":"node_sequence_air",
                                                                        "Passengers":"trips_air"
                                                                        })

        trips_with_complete_path=pd.concat([trips_incoming_with_complete_path,trips_outgoing_with_complete_path])

        #re-order the columns
        trips_with_complete_path=trips_with_complete_path[["origin",
                                                                "destination",
                                                                "node_sequence_ground",
                                                                "node_sequence_air",
                                                                "trips",
                                                                "trips_air",
                                                                "cost_air",
                                                                "distance_air"
                                                                ]]
        return trips_with_complete_path 

In [167]:
thing=list(coeff_incoming.columns.values)

In [168]:
trips_complete_small=assign_ground_path(aw_paths_w_costs_small,coeff_outgoing,coeff_incoming)

C:\Users\LMENENDEZ\AppData\Local\Temp\ipykernel_10928\3886800579.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trips_incoming_with_complete_path=pd.concat([trips_incoming_with_complete_path,new_df])
C:\Users\LMENENDEZ\AppData\Local\Temp\ipykernel_10928\3886800579.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trips_outgoing_with_complete_path=pd.concat([trips_outgoing_with_complete_path,new_df])


In [169]:
trips_complete_small

,origin,destination,node_sequence_ground,node_sequence_air,trips,trips_air,cost_air,distance_air
0,BRUSSELS,Badajoz,egress,"['BRUSSELS', 'LEZL']",1.0,75.862333,150.98,1709.000000
1,BRUSSELS,Cádiz,egress,"['BRUSSELS', 'LEMD', 'LEZL']",1.0,11.181667,150.98,1709.000000
2,BRUSSELS,Cádiz,"['007151003', '007151406']","['BRUSSELS', 'LEZL']",1.0,75.862333,150.98,1709.000000
3,BRUSSELS,Cádiz,egress,"['BRUSSELS', 'LEZL']",3.0,75.862333,150.98,1709.000000
4,BRUSSELS,Córdoba,egress,"['BRUSSELS', 'LEMD', 'LEZL']",2.0,11.181667,150.98,1709.000000
...,...,...,...,...,...,...,...,...
235,Sevilla,LPPR,access,"['LEZL', 'LEMD', 'LPPR']",12.0,11.506333,102.61,640.333333
236,Sevilla,LPPR,access,"['LEZL', 'LPPR']",69.0,76.320000,102.61,640.333333
237,Sevilla,LPPT,access,"['LEZL', 'LPPT']",40.0,44.416333,133.41,322.000000
238,Sevilla,LSGG,access,"['LEZL', 'LSGG']",67.0,73.169333,91.86,1394.000000


In [170]:
trips_complete_small[trips_complete_small["trips_air"]==75.862333]

,origin,destination,node_sequence_ground,node_sequence_air,trips,trips_air,cost_air,distance_air
0,BRUSSELS,Badajoz,egress,"['BRUSSELS', 'LEZL']",1.0,75.862333,150.98,1709.0
2,BRUSSELS,Cádiz,"['007151003', '007151406']","['BRUSSELS', 'LEZL']",1.0,75.862333,150.98,1709.0
3,BRUSSELS,Cádiz,egress,"['BRUSSELS', 'LEZL']",3.0,75.862333,150.98,1709.0
6,BRUSSELS,Sevilla,egress,"['BRUSSELS', 'LEZL']",71.0,75.862333,150.98,1709.0


In [171]:
trips_complete_small[trips_complete_small["trips_air"]==76.320000]

,origin,destination,node_sequence_ground,node_sequence_air,trips,trips_air,cost_air,distance_air
80,Cádiz,LPPR,access,"['LEZL', 'LPPR']",1.0,76.32,102.61,640.333333
113,Córdoba,LPPR,access,"['LEZL', 'LPPR']",1.0,76.32,102.61,640.333333
169,Huelva,LPPR,access,"['LEZL', 'LPPR']",4.0,76.32,102.61,640.333333
181,Málaga,LPPR,access,"['LEZL', 'LPPR']",1.0,76.32,102.61,640.333333
236,Sevilla,LPPR,access,"['LEZL', 'LPPR']",69.0,76.32,102.61,640.333333


In [172]:
trips_complete=assign_ground_path(trips_aw=aw_paths_w_costs_cities,
                                  coeff_incoming=coeff_incoming,
                                  coeff_outgoing=coeff_outgoing
                                  )

C:\Users\LMENENDEZ\AppData\Local\Temp\ipykernel_10928\3886800579.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trips_incoming_with_complete_path=pd.concat([trips_incoming_with_complete_path,new_df])
C:\Users\LMENENDEZ\AppData\Local\Temp\ipykernel_10928\3886800579.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trips_outgoing_with_complete_path=pd.concat([trips_outgoing_with_complete_path,new_df])


In [178]:
print(trips_complete.shape[0])
print(trips_complete[trips_complete["trips"]==1].shape[0])
print(trips_complete[trips_complete["trips"]==2].shape[0])
print(trips_complete[trips_complete["trips"]==3].shape[0])

13454
5971
1929
856
